# Bringing Functional Programming into an Imperative World.
# Maybe


<img src="Pell_Contact.png">


# Why FP?


* Expressive
* Efficient
* Safe-er
* Easier to work with concurrent programming


# Higher order functions


* Functions get treated like first class citizens
    * Passed into other functions as parameters
    * Returned from functions as values
    * Stored in variables

### Passing in a function

In [ ]:
def call_db(fn):
    print("Initializing database connection")
    curs = "cursor"
    return fn(curs)
    
def query(curs):
    results = "Query db using {}".format(curs)
    return results

def insert_into_db(curs):
    new_row_number = "Insert item into db using {}".format(curs)
    return new_row_number


In [ ]:
res = call_db(query)
print(res)

In [ ]:
new_row = call_db(insert_into_db)
print(new_row)

### Returning a function

In [ ]:
def complex_string_fn(s):
    print("Hold up, I'm doing something complex with your {}". format(s))
    
def complex_list_fn(l):
    print("Hold up, I'm doing something complex with your {}". format(l))
    
def give_me_a_fn(source):
    if source == "s3":
        return complex_string_fn
    else:
        return complex_list_fn
    

In [ ]:
source = "s3"
new_fn = give_me_a_fn(source)
new_fn("S3 object")

In [ ]:
source = "sql"
new_fn = give_me_a_fn(source)
new_fn("MySQL object")

In [ ]:
source = "sql"
give_me_a_fn(source)("Postgres Object")

### Decorators

In [ ]:
def decorate_it(fn):
    def wrapper(*args):
        fn(*args)
        print("Consider yourself decorated")
    return wrapper

In [ ]:
@decorate_it
def add_it(x, y):
    print("It's gonna equal: ", x + y)

In [ ]:
add_it(3, 5)

# Immutable Data

* The structure itself never gets changed
* New way of thinking about "updating" your data structure
    * every update returns a new data structure
* Safety
    * since each update returns a new data structure, you can safely pass them around
* Historically inefficient, but that's changing
    * [Persistent](https://en.wikipedia.org/wiki/Persistent_data_structure) data structures are beginning to make thier way into languages

### The problem with mutable data structures
* You can't be sure that your data won't change out from under you

In [ ]:
list_1 = [1, 2, 3, 4]
list_2 = list_1
list1 = list_1.append(5)

print(list_1)
print(list_2)
print(list_1 is list_2)

### Immutable data structures help avoid this problem
* Data structures that cannot have their values changed
* Python has a few immutable types, the most common is the tuple

In [ ]:
x = 1, 2, "me"

# adding to a tuple is slow because it has to copy the entire tuple
y = (4,) + x
print(x)
print(y)

### You can fake an immutable data structure using deepcopy
* But it's still slow

In [ ]:
from copy import deepcopy
list_1 = [1, 2, 3,4]
list_2 = list_1
print(list_1)
print(list_2)
print(list_1 is list_2)
print()

list_2 = deepcopy(list_1)
list_1.append(5)
print(list_1)
print(list_2)
print(list_1 is list_2)

# Recursion

* Calling a function from inside itself. 

* Recursion takes the place of loops in many functional languages

* Every time you call, you add to the stack

* Tail call optimizations eliminate this if you design your function properly
    * But, python doesn't have TCO


### What's wrong with loops?

In [ ]:
x = [1, 2, 3]
for a in x:
    print(a)
    x.append(a + 1)
    if len(x) > 100:
        break

### Let's look at recursion

In [ ]:
def factorial( n ):
   if n <1:   # base case
       return 1
   else:
       returnNumber = n * factorial( n - 1 )  # recursive call
       print(str(n) + '! = ' + str(returnNumber))
       return returnNumber
    
factorial(5)

### what the stack sees


5 \* factorial(4)

5 \* factorial(4) \* factorial(3)

5 \* factorial(4) \* factorial(3) \* factorial(2)

5 \* factorial(4) \* factorial(3) \* factorial(2) \* factorial(1)

5 \* factorial(4) \* factorial(3) \* factorial(2) \* 1

5 \* factorial(4) \* factorial(3) \* 2 

5 \* factorial(4) \* 6

5 \* 24

120


### The problem with recursion in python

In [ ]:
def factorial( n ):
   if n <1:   # base case
       return 1
   else:
       returnNumber = n * factorial( n - 1 )  # recursive call
       print(str(n) + '! = ' + str(returnNumber))
       return returnNumber
    
factorial(1000)

# Function Composition

* Combine functions just like you would in math
* The result of one function is passed in as the parameter to the next
* f(g(x))


In [102]:
def get_users():
    users = {1: ["Rob", 25], 2: ["Jon", 21], 3: ["Arya", 15], 4: ["Sansa", 18], 5: ["Bran", 13] }
    return users

def get_18_and_over(users):
    adults = {}
    for k, v in users.items():
        if v[1] >= 18:
            adults[k] = v
    return adults



In [103]:
adults = get_18_and_over(get_users())
print(adults)

{1: ['Rob', 25], 2: ['Jon', 21], 4: ['Sansa', 18]}


# Map, Filter, Reduce, Lambdas

## Lambdas

* Anonymous functions
    * This just means they don't have a name

In [108]:
sum = lambda x, y: x + y

sum(2,3)

5

## Filter

In [106]:
list_1 = [1, 2, 3, 4]

In [107]:
list_3 = list(filter(lambda x: x % 2 == 0, list_1))
print(list_1)
print(list_3)

[1, 2, 3, 4]
[2, 4]


## Map

* Takes a mapping function and a collection to operate on
* Applies the function to each item in the list, returning an iterator over the new list

In [109]:
list(map(lambda x: x *2, list_1))

[2, 4, 6, 8]

## Reduce

* Takes a reducing function and a collection to reduce and an optional initial value
    * If there's no initial value, it passes the first two collection items into the function
* The function needs to take 2 parameters and return some combination of the two in the same form as the parameters

In [127]:
from functools import reduce
list_1 = [1, 2, 3, 4]
reduce(lambda x, y: x * y, list_1)

24

In [129]:
dna = ["AAACTCTGGT", "AACTGGTC", "CCCTGTGT"]

a_count = reduce(lambda a, x: a + x.count("A"), dna, 0)
a_count

5

In [125]:
def combined_age(person1, person2):
    age1 = person1[1]
    age2 = person2[1]
    c_age = ["combined", age1 + age2]
    return c_age

users = get_users()

combined = reduce(combined_age, users.values())
print(combined)

['combined', 92]


## you can mimic map and filter in list comprehensions

### map

In [ ]:
list_2 =[(lambda x: x*x)(x) for x in list_1]
print(list_2)

### or if the lambda is too ugly

In [ ]:
def square(x):
    return x*x

[square(x) for x in list_1]



### filter


In [ ]:
[x for x in list_1 if x % 2 == 0]